In [1]:
import sys
import os
import glob
#print(sys.version_info)
import pandas as pd
import numpy as np
from sklearn import metrics,preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import collections

# Imports for PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
# getting the labelled data and features from tran_feature_selection notebook
%store -r mixed_flows
%store -r features

%store -r benign_flows

In [3]:
x = benign_flows.loc[:, features].values
x = StandardScaler().fit_transform(x) # normalizing the features
np.mean(x),np.std(x)

(2.223487803925044e-16, 1.0000000000000013)

In [4]:
feat_cols = ['feature'+str(i) for i in range(x.shape[1])]
normalised_benign_flows = pd.DataFrame(x,columns=feat_cols)
# normalised_benign_flows.tail(5)

In [8]:
# finding the top k components whose cumulative variance exceeds 95%

for i in range(2,len(features)):
#     print(i)
    pca_benign = PCA(n_components=i) 
    principalComponents_benign = pca_benign.fit_transform(x)
    var=np.cumsum(np.round(pca_benign.explained_variance_ratio_, decimals=3)*100)
    array_length = len(var)
    last_element = var[array_length - 1]
    if last_element>95:
        top_k_component = i
        break
        
print(top_k_component)

34


In [6]:
# plotting the principal component

# plt.figure(figsize=(16,10))
# sns.scatterplot(
#     x="principal component 1", y="principal component 2",
#     palette=sns.color_palette("hls", 10),
#     data=principal_benign_Df,
#     legend="full",
#     alpha=0.3
# )

In [9]:
# re-run PCA with only top k features again
top_k_features = features[:top_k_component-1]

top_k_x = benign_flows.loc[:, top_k_features].values
top_k_x = StandardScaler().fit_transform(top_k_x) # normalizing the features
np.mean(top_k_x),np.std(top_k_x)

top_k_feat_cols = ['feature'+str(i) for i in range(top_k_x.shape[1])]

top_k_pca = PCA(n_components=top_k_component-1)   
top_k_principalComponents = top_k_pca.fit_transform(top_k_x)

top_k_var=np.cumsum(np.round(top_k_pca.explained_variance_ratio_, decimals=3)*100)
print(top_k_var)


[ 13.5  26.1  33.4  40.2  45.8  50.9  55.7  60.1  63.7  66.9  69.9  72.9
  75.9  78.8  81.6  84.1  86.4  88.7  90.9  93.   94.5  95.9  97.1  98.
  98.7  99.3  99.6  99.8  99.9 100.  100.  100.  100. ]
